## 准备工作
从 https://www.kaggle.com/datasets/dylanjcastillo/news-headlines-2024/ 下载新闻数据集，news_data_dedup.csv 并将其放入当前目录。

In [ ]:
!pip install "pymilvus[model]"
!pip install hdbscan
!pip install plotly
!pip install umap-learn

## 提取 Embeddings 至 Milvus
我们将使用 Milvus 创建一个 Collections，并使用 BGE-M3 模型提取密集嵌入。

In [ ]:
import pandas as pd
from dotenv import load_dotenv
from pymilvus.model.hybrid import BGEM3EmbeddingFunction
from pymilvus import FieldSchema, Collection, connections, CollectionSchema, DataType
from modelscope import snapshot_download

load_dotenv()

df = pd.read_csv("news_data_dedup.csv")

# 使用列表推导式，将dataFame的title和description列拼接成一个列表，每个字符串由标题和描述组成
docs = [
    f"{title}\n{description}" for title, description in zip(df.title, df.description)
]

# 使用modelscope下载模型
model_path = snapshot_download('BAAI/bge-m3', revision='master')
# 然后使用本地路径加载模型
ef = BGEM3EmbeddingFunction(
    model_name=model_path,  # 使用本地模型路径
    device='cpu', # 指定设备为cpu
    use_fp16=False # 是否使用fp16精度
)

embeddings = ef(docs)["dense"] # ef为嵌入函数，docs为输入数据，dense键对应的值为嵌入向量

connections.connect(uri="milvus.db") # 数据存入

In [ ]:
# 结构
fields = [
    FieldSchema(
        name="id", dtype=DataType.INT64, is_primary=True, auto_id=True
    ), 
    FieldSchema(
        name="embedding", dtype=DataType.FLOAT_VECTOR, dim=1024
    ), 
    FieldSchema(
        name="text", dtype=DataType.VARCHAR, max_length=65535
    ), 
]

schema = CollectionSchema(fields=fields, description="Embedding collection")

collection = Collection(name="news_data", schema=schema)

for doc, embedding in zip(docs, embeddings):
    collection.insert({"text": doc, "embedding": embedding})
    print(doc)

index_params = {"index_type": "FLAT", "metric_type": "L2", "params": {}}

collection.create_index(field_name="embedding", index_params=index_params)

collection.flush()

## 为 HDBSCAN 构建距离矩阵
HDBSCAN 需要计算点与点之间的距离来进行聚类，计算量很大。由于远处的点对聚类分配的影响较小，我们可以通过计算前 k 个近邻来提高效率。在本例中，我们使用的是 FLAT 索引，但对于大规模数据集，Milvus 支持更高级的索引方法来加速搜索过程。 首先，我们需要获取一个迭代器来迭代之前创建的 Milvus Collections。

In [ ]:
import hdbscan
import numpy as np
import pandas as pd
import plotly.express as px
from umap import UMAP
from pymilvus import Collection

collection = Collection(name="news_data")
collection.load()

# 查询迭代器
# batch_size为每次查询的条数
# expr为查询条件
# output_fields为输出的字段
iterator = collection.query_iterator(
    batch_size=10, expr="id > 0", output_fields=["id", "embedding"]
)

# L2表示欧氏距离 作为相似性度量方法
# nprobe表示查询时的探测数 决定了搜索的精度和速度
search_params = {
    "metric_type": "L2",
    "params": {"nprobe": 10},
}  
ids = []
dist = {}

embeddings = []

2025-04-02 15:55:55,466 [WARNING][__setup_ts_by_request]: failed to get mvccTs from milvus server, use client-side ts instead (iterator.py:258)


## 聚类操作
下面我们来演示如何使用HDBScan进行聚类操作。

In [ ]:
while True:
    # 提取每条数据的id和embedding，id存入ids列表中
    batch = iterator.next()
    batch_ids = [data["id"] for data in batch]
    ids.extend(batch_ids)
    # 将每条数据的embedding存入embeddings列表中 
    query_vectors = [data["embedding"] for data in batch]
    embeddings.extend(query_vectors)
    # 现在获取到了ids列表和embeddings列表，我们可以使用hdbscan进行聚类操作了


    # 使用milvus的搜索
    results = collection.search(
        data=query_vectors,
        limit=50,
        anns_field="embedding",
        param=search_params,
        output_fields=["id"],
    )
    # 搜索结果存入dist字典中，键为batch_id，值为一个列表，列表中每个元素为一个元组，元组的第一个元素为id，第二个元素为距离（与该向量最相似的ID及其距离）
    for i, batch_id in enumerate(batch_ids):
        dist[batch_id] = []
        for result in results[i]:
            dist[batch_id].append((result.id, result.distance))

    if len(batch) == 0:
        break

# 构建距离矩阵
# ids2index是一个字典，键为id，值为该id在ids列表中的索引
ids2index = {}

for id in dist:
    ids2index[id] = len(ids2index)

# dist_metric是二维距离矩阵
dist_metric = np.full((len(ids), len(ids)), np.inf, dtype=np.float64)

# 根据字典中的搜索结果填充距离矩阵，表示meita_id和batch_id之间的距离
for id in dist:
    for result in dist[id]:
        dist_metric[ids2index[id]][ids2index[result[0]]] = result[1]

# 使用HDBSCAN进行聚类，min_samples为每个点的最小邻居数，min_cluster_size为每个簇的最小点数，metric为距离度量方法，precomputed表示使用预计算的距离矩阵
h = hdbscan.HDBSCAN(min_samples=3, min_cluster_size=3, metric="precomputed")
hdb = h.fit(dist_metric)

之后，HDBSCAN 聚类就完成了。我们可以获取一些数据并显示其聚类结果。请注意，有些数据不会被分配到任何聚类中，这意味着它们是噪音，因为它们位于某些稀疏区域。

## 使用 UMAP 进行聚类可视化
我们已经使用 HDBSCAN 对数据进行了聚类，并获得了每个数据点的标签。不过，利用一些可视化技术，我们可以获得聚类的全貌，以便进行直观分析。现在，我们将使用 UMAP 对聚类进行可视化。UMAP 是一种用于降维的高效方法，它在保留高维数据结构的同时，将其投影到低维空间，以便进行可视化或进一步分析。有了它，我们就能在二维或三维空间中可视化原始高维数据，并清楚地看到聚类。 在这里，我们再次遍历数据点，获取原始数据的 ID 和文本，然后使用 ploty 将数据点与这些元信息绘制成图，并用不同的颜色代表不同的聚类。

In [ ]:
import plotly.io as pio

# 设置plotly的渲染器为notebook
pio.renderers.default = "notebook"

# 创建一个UMAP对象
# 用于将高维数据降维到二维空间。UMAP的参数包括：
# n_components=2：降维到二维。
# random_state=42：设置随机种子以确保结果可复现。
# n_neighbors=80和min_dist=0.1：控制UMAP的局部和全局结构保留程度
umap = UMAP(n_components=2, random_state=42, n_neighbors=80, min_dist=0.1)


df_umap = (
    # 创建降维后的DataFrame
    # 使用UMAP对embeddings（假设是高维嵌入向量）进行降维，并将结果存储在一个Pandas DataFrame中，列名为x和y。随后：
    # 添加一列cluster，其值为HDBSCAN聚类结果的标签（hdb.labels_）。
    # 过滤掉噪声点（cluster == "-1"）。
    # 按照cluster列对数据进行排序。
    pd.DataFrame(umap.fit_transform(np.array(embeddings)), columns=["x", "y"])
    .assign(cluster=lambda df: hdb.labels_.astype(str))
    .query('cluster != "-1"')
    .sort_values(by="cluster")
)
# 从milvus中批量的查询数据
iterator = collection.query_iterator(
    batch_size=10, expr="id > 0", output_fields=["id", "text"]
)

ids = []
texts = []
# 查询到的数据存储到列表中
while True:
    batch = iterator.next()
    if len(batch) == 0:
        break
    batch_ids = [data["id"] for data in batch]
    batch_texts = [data["text"] for data in batch]
    ids.extend(batch_ids)
    texts.extend(batch_texts)

show_texts = [texts[i] for i in df_umap.index]

df_umap["hover_text"] = show_texts
fig = px.scatter(
    df_umap, x="x", y="y", color="cluster", hover_data={"hover_text": True}
)
fig.show()